# Material de estudo para Constraint Programming e modelagem
## Daniel Falci

### Contexto

Considere uma empresa hipotética onde um grupo de colaboradores candidatos $N$ é formado pelos colaboradores ${\{n_1, n_2, ..., n_i\}}$. Cada um deles passa por uma avaliação periódica de desempenho quando lhes é conferido um valor contínuo entre 0 e 5 para um rol de competências necessárias ao desempenho de sua função. A lista de competências avaliadas é ilustrada abaixo:

- Conhecimento Técnico
  - Java
  - Python
  - C++
  - ADVPL
  - Javascript
- Cultura organizacional
  - Comprometimento
  - Organização
  - Métodos Ageis
- Social
  - Empatia
  - Comportamento em grupo

Cada colaborador tem um custo associado de forma individualizada e mensurado em unidades de valor por hora. A empresa trabalha com projetos e, neste caso, o líder de um projeto é o responsável por indicar as competências mínimas necessárias à sua execução, bem como a quantidade de integrantes a serem utilizados. 

### Objetivo

De posse da quantidade de colaboradores e competências mínimas necessárias em um projeto, a empresa deseja identificar quais funcionários devem atuar de forma a minimizar o custo do projeto e maximizar a competência da equipe escollhida. A relação entre custo e competência é ponderada, de forma que o líder do projeto pode decidir por privilegiar alguma competência em detrimento do custo ou vice-versa.

<!-- $$ (min \sum c_{n_i}),  (max \sum{hab_{n_i}})$$ -->

escrever formula depois



### Resolução

Primeiro vamos instalar as bibliotecas necessárias para executar o sistema

In [1]:
!pip install ortools

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### DEFINIÇÃO DOS COLABORADORES POSSÍVEIS E DAS NECESSIDADES DO PROJETO

In [2]:
# declaracao das pessoas passiveis de serem utilizadas no projeto
pessoas = [
    {
        'nome':'pessoa_a',
        'custo_por_hora':75,
        'hab_tecnicas':{
            'java':5,
            'python':4.5,
            'c++':4,
            'advlp':1,
            'javascript':5
        },
        'hab_culturais':{
            'organizacao':3,
            'metodos_ageis':2,
            'comprometimento':5
        },
        'hab_sociais':{
            'empatia':3,
            'outra':2
        }
    },

    {
        'nome':'pessoa_d',
        'custo_por_hora':25,
        'hab_tecnicas':{
            'java':2,
            'python':1,
            'c++':1,
            'advlp':3,
            'javascript':2
        },
        'hab_culturais':{
            'organizacao':2,
            'metodos_ageis':2,
            'comprometimento':2
        },
        'hab_sociais':{
            'empatia':2,
            'outra':2
        }
    },

    {
        'nome':'pessoa_b',
        'custo_por_hora':50,
        'hab_tecnicas':{
            'java':3,
            'python':1,
            'c++':1,
            'advlp':5,
            'javascript':3
        },
        'hab_culturais':{
            'organizacao':3,
            'metodos_ageis':5,
            'comprometimento':3
        },
        'hab_sociais':{
            'empatia':2,
            'outra':3
        }
    },

    {
        'nome':'pessoa_c',
        'custo_por_hora':80,
        'hab_tecnicas':{
            'java':3,
            'python':1,
            'c++':1,
            'advlp':5,
            'javascript':4
        },
        'hab_culturais':{
            'organizacao':4,
            'metodos_ageis':5,
            'comprometimento':4
        },
        'hab_sociais':{
            'empatia':2,
            'outra':4
        }
    },
]

# necessidades do projeto em si
necessidade_projeto = {
    'hab_tecnicas':{
        'java':2,
        'advpl':1,
        'javascript':1
    },
    'hab_culturais':{
        'organizacao':2,
        'comprometimento':2
    },
    'hab_sociais':{
        'outra':1
    }
}



### MODELAGEM DO PROBLEMA USANDO CONSTRAINT PROGRAMMING (IP)




In [3]:
class Solucao():
    """
    Coleta a melhor solucao possivel na otimizacao
    """
    def __init__(self, iteracao, pessoas):
        self.pessoas = [pessoa for pessoa in pessoas if pessoa['var'].Value() == 1 ]
        self.iteracao = iteracao


    def get_custo(self):
        return sum([pessoa['custo_por_hora'] for pessoa in self.pessoas]), sum([pessoa['tot_tecnica'] for pessoa in self.pessoas])

    def get_pessoas(self):
        return [pessoa['nome'] for pessoa in self.pessoas]

    def __repr__(self):
        valor, tecnica = self.get_custo()
        return 'Pessoas selecionadas : {}\nCusto monetario da solucao : {}\nCoeficiente tecnico : {}'.format(self.get_pessoas(), valor, tecnica)

def preparar_dados(pessoas):
    """
    Passo necessario para otimizacao multiobjetivo : (otimizacao por maximizacao)
    - Inverte a o custo por hora, faz a normalizacao e deixa na escala de inteiros de 0 - 100. Desta forma, aquele que custa menos tera um maior valor (proximo de 100) e o que custa mais tera um valor de 0.
    - Totaliza e normaliza cada uma das listas de habilidades com notas de 0 a 5
    :param pessoas:
    :return:
    """
    valor = max([pessoa['custo_por_hora'] for pessoa in pessoas])
    for pessoa in pessoas:
        pessoa['custo_convertido'] = int(((valor - pessoa['custo_por_hora'])/float(valor)) * 100)

        pessoa['tot_tecnica'] = (sum(pessoa['hab_tecnicas'][key] for key in pessoa['hab_tecnicas']) / (len(pessoa['hab_tecnicas']) * 5.)) * 100
        pessoa['tot_cultura'] = (sum(pessoa['hab_culturais'][key] for key in pessoa['hab_culturais'])/ (len(pessoa['hab_culturais']) * 5.)) * 100
        pessoa['tot_soc'] = (sum(pessoa['hab_sociais'][key] for key in pessoa['hab_sociais'])/ (len(pessoa['hab_sociais']) * 5.)) * 100

def resolver_otimizacao(necessidade_projeto, pessoas, membros_necessarios, coeficientes=[40, 60]):
    from ortools.constraint_solver import pywrapcp
    solver = pywrapcp.Solver("Teste")

    # declarando as variaveis
    for i in xrange(0, len(pessoas)):
        pessoas[i]['var'] = solver.IntVar(0, 1, 'qt_{}'.format(pessoas[i]['nome']))


    # declarando restricoes
    for hab in necessidade_projeto:
        for chave_hab in necessidade_projeto[hab]:
            valor_minimo = necessidade_projeto[hab][chave_hab]
            for pessoa in pessoas:
                if not pessoa.get(hab, None) is None and not pessoa[hab].get(chave_hab, None) is None:
                    # garante que a habilidade da pessoa atenda ao projeto
                    solver.Add(solver.Max(
                        pessoa['var'] == 0,
                        pessoa['var'] * pessoa[hab][chave_hab] >= valor_minimo
                    ) == True)

    # garante que a quantidade de pessoas vai ser exatamente igual a quantidade de membros necessarios no projeto
    solver.Add(sum([pessoa['var'] for pessoa in pessoas]) == membros_necessarios)

    #funcao objetivo
    objetivo = solver.Maximize(
        (coeficientes[0] * solver.Sum([pessoa['var'] * pessoa['custo_convertido'] for pessoa in pessoas])) +
        (coeficientes[1] * solver.Sum([pessoa['var'] * int(pessoa['tot_tecnica']) for pessoa in pessoas])),
    1)




    db = solver.Phase([pessoa['var'] for pessoa in pessoas], solver.CHOOSE_MIN_SIZE_LOWEST_MAX, solver.ASSIGN_CENTER_VALUE)
    solver.TimeLimit(30000)#quero a melhor solucao com 30 segundos de computacao


    solver.NewSearch(db, [objetivo])
    num_solutions = 0

    ultimaSolucao = None

    while solver.NextSolution():
        num_solutions += 1
        print [pessoa['var'] for pessoa in pessoas]
        ultimaSolucao = Solucao(num_solutions, pessoas)

    solver.EndSearch()

    if ultimaSolucao!= None:
        print '\n\nMELHOR SOLUCAO ENCONTRADA'
        print ultimaSolucao
    else:
        print 'NENHUMA SOLUCAO ENCONTRADA'
        
    print '\n'

    print num_solutions
    print 'falhas : ', solver.Failures()
    print 'branches : ',solver.Branches()
    print 'tempo em ms : ',solver.WallTime()

# normaliza e prepara os colaboradores para o metodo
preparar_dados(pessoas)

resolver_otimizacao(necessidade_projeto, pessoas, 2, coeficientes=[40, 60])

[qt_pessoa_a(0), qt_pessoa_d(0), qt_pessoa_b(1), qt_pessoa_c(1)]
[qt_pessoa_a(0), qt_pessoa_d(1), qt_pessoa_b(0), qt_pessoa_c(1)]
[qt_pessoa_a(0), qt_pessoa_d(1), qt_pessoa_b(1), qt_pessoa_c(0)]
[qt_pessoa_a(1), qt_pessoa_d(0), qt_pessoa_b(1), qt_pessoa_c(0)]
[qt_pessoa_a(1), qt_pessoa_d(1), qt_pessoa_b(0), qt_pessoa_c(0)]


MELHOR SOLUCAO ENCONTRADA
Pessoas selecionadas : ['pessoa_a', 'pessoa_d']
Custo monetario da solucao : 100
Coeficiente tecnico : 114.0


5
falhas :  1
branches :  10
tempo em ms :  5


#### Testando com dados mais volumosos (mas nem tanto)




In [4]:
def gerar_pessoas_aleatorias(n=100):
    from random import randint
    arr = [
        {
            'nome':'pessoa_{}'.format(i),
            'custo_por_hora':randint(25, 270),
            'hab_tecnicas':{
                'java':randint(0, 5),
                'python':randint(0, 5),
                'c++':randint(0, 5),
                'advlp':randint(0, 5),
                'javascript':randint(0, 5)
            },
            'hab_culturais':{
                'organizacao':randint(0, 5),
                'metodos_ageis':randint(0, 5),
                'comprometimento':randint(0, 5)
            },
            'hab_sociais':{
                'empatia':randint(0, 5),
                'outra':randint(0, 5)
            }
        } for i in xrange(n)
    ]
    preparar_dados(arr)
    return arr

  
print '#### PROBLEMA MAIOR ###'

# agora um problema maior
pessoas = gerar_pessoas_aleatorias(100)
nova_necessidade = {
    'hab_tecnicas':{
        'java':3,
        'javascript':3
    },
    'hab_culturais':{
        'organizacao':2,
        'comprometimento':2
    },
    'hab_sociais':{
        'outra':2
    }
}

resolver_otimizacao(nova_necessidade, pessoas, 5, coeficientes=[40, 60])

#### PROBLEMA MAIOR ###
[qt_pessoa_0(0), qt_pessoa_1(0), qt_pessoa_2(0), qt_pessoa_3(0), qt_pessoa_4(0), qt_pessoa_5(0), qt_pessoa_6(0), qt_pessoa_7(0), qt_pessoa_8(0), qt_pessoa_9(0), qt_pessoa_10(0), qt_pessoa_11(0), qt_pessoa_12(0), qt_pessoa_13(0), qt_pessoa_14(0), qt_pessoa_15(0), qt_pessoa_16(0), qt_pessoa_17(0), qt_pessoa_18(0), qt_pessoa_19(0), qt_pessoa_20(0), qt_pessoa_21(0), qt_pessoa_22(0), qt_pessoa_23(0), qt_pessoa_24(0), qt_pessoa_25(0), qt_pessoa_26(0), qt_pessoa_27(0), qt_pessoa_28(0), qt_pessoa_29(0), qt_pessoa_30(0), qt_pessoa_31(0), qt_pessoa_32(0), qt_pessoa_33(0), qt_pessoa_34(0), qt_pessoa_35(0), qt_pessoa_36(0), qt_pessoa_37(0), qt_pessoa_38(0), qt_pessoa_39(0), qt_pessoa_40(0), qt_pessoa_41(0), qt_pessoa_42(0), qt_pessoa_43(0), qt_pessoa_44(0), qt_pessoa_45(0), qt_pessoa_46(0), qt_pessoa_47(0), qt_pessoa_48(0), qt_pessoa_49(0), qt_pessoa_50(0), qt_pessoa_51(0), qt_pessoa_52(0), qt_pessoa_53(0), qt_pessoa_54(0), qt_pessoa_55(0), qt_pessoa_56(0), qt_pessoa_57(0),

## Complicando o problema um pouquinho mais

Imagine agora que, além das competências mínimas do projeto, temos à disposição a produtividade necessária (em story points por hora) para cumprí-lo no prazo

Aos colaboradores é atribuída uma medida de produtividade individual (story points por hora) e a produtividade do grupo é dada pelo somatório da produtividade dos indivíduos do grupo.

A idéia agora é determinar quais colaboradores (e quantos por conseguinte) serão necessários para atender a produtividade necessária do projeto e as sus competências mínimas de forma a minimizar o custo do projeto e maximizar a competência da equipe selecionada.

#### Mudanças em relação à modelagem anterior

* Cada colaborador recebe um novo atributo, denominado 'produtividade' - Produtividade por hora do indivíduo
* Removemos o argumento membros_necessarios. Ao contrário do passo anterior,  desta vez cabe ao sistema definir a quantidade de colaboradores a serem utilizados.
* Invertemos a função objetivo: Antes maximizava "a pechincha" e a competência do grupo selecionado. Agora minimiza o custo e o déficit de competência da equipe.
* Uma das restrições é alterada. Agora, o somatório da produtividade do grupo de individuos na solução deve ser igual ou superior ao valor requerido para a conclusão do projeto. 


Veja a implementação do modelo abaixo.

In [5]:
# declaracao das pessoas passiveis de serem utilizadas no projeto
pessoas = [
    {
        'nome':'pessoa_a',
        'produtividade':12,
        'custo_por_hora':113,
        'hab_tecnicas':{
            'java':5,
            'python':4.5,
            'c++':4,
            'advlp':1,
            'javascript':5
        },
        'hab_culturais':{
            'organizacao':3,
            'metodos_ageis':2,
            'comprometimento':5
        },
        'hab_sociais':{
            'empatia':3,
            'outra':2
        }
    },

    {
        'nome':'pessoa_d',
        'produtividade':6,
        'custo_por_hora':25,
        'hab_tecnicas':{
            'java':2,
            'python':1,
            'c++':1,
            'advlp':3,
            'javascript':2
        },
        'hab_culturais':{
            'organizacao':2,
            'metodos_ageis':2,
            'comprometimento':2
        },
        'hab_sociais':{
            'empatia':2,
            'outra':2
        }
    },

    {
        'nome':'pessoa_b',
        'produtividade':7,
        'custo_por_hora':50,
        'hab_tecnicas':{
            'java':3,
            'python':1,
            'c++':1,
            'advlp':5,
            'javascript':3
        },
        'hab_culturais':{
            'organizacao':3,
            'metodos_ageis':5,
            'comprometimento':3
        },
        'hab_sociais':{
            'empatia':2,
            'outra':3
        }
    },

    {
        'nome':'pessoa_c',
        'custo_por_hora':80,
        'produtividade':10,
        'hab_tecnicas':{
            'java':3,
            'python':1,
            'c++':1,
            'advlp':5,
            'javascript':4
        },
        'hab_culturais':{
            'organizacao':4,
            'metodos_ageis':5,
            'comprometimento':4
        },
        'hab_sociais':{
            'empatia':2,
            'outra':4
        }
    },
]

# necessidades do projeto em si
necessidade_projeto = {
    'hab_tecnicas':{
        'java':3,
        'advpl':1,
        'javascript':1
    },
    'hab_culturais':{
        'organizacao':1,
    }
}

def preparar_dados(pessoas):
    """
    Passo necessario para otimizacao multiobjetivo : (otimizacao por minimizacao)
    - faz a normalizacao do custo e deixa na escala de inteiros de 0 - 100. Desta forma, aquele que custa mais tera um maior valor (proximo de 100) e o que custa menos tera um valor de 0.
    - Totaliza e normaliza cada uma das listas de habilidades com notas de 0 a 5
    :param pessoas:
    :return:
    """
    valor = max([pessoa['custo_por_hora'] for pessoa in pessoas])
    for pessoa in pessoas:
        pessoa['custo_convertido'] = int(((pessoa['custo_por_hora'])/float(valor)) * 100)

        pessoa['tot_tecnica'] = 100 - (sum(pessoa['hab_tecnicas'][key] for key in pessoa['hab_tecnicas']) / (len(pessoa['hab_tecnicas']) * 5.)) * 100
        pessoa['tot_cultura'] = 100 - (sum(pessoa['hab_culturais'][key] for key in pessoa['hab_culturais'])/ (len(pessoa['hab_culturais']) * 5.)) * 100
        pessoa['tot_soc'] = 100 -(sum(pessoa['hab_sociais'][key] for key in pessoa['hab_sociais'])/ (len(pessoa['hab_sociais']) * 5.)) * 100

# normaliza e prepara os colaboradores para o metodo
preparar_dados(pessoas)




In [6]:
class Solucao():
    """
    Coleta a melhor solucao possivel na otimizacao
    """
    def __init__(self, iteracao, pessoas):
        self.pessoas = [pessoa for pessoa in pessoas if pessoa['var'].Value() == 1 ]
        self.iteracao = iteracao


    def get_custo(self):
        return sum([pessoa['custo_por_hora'] for pessoa in self.pessoas]), sum([(pessoa['tot_tecnica'] - 100)*-1 for pessoa in self.pessoas]), sum([pessoa['produtividade'] for pessoa in self.pessoas])

    def get_pessoas(self):
        return [pessoa['nome'] for pessoa in self.pessoas]

    def __repr__(self):
        valor, tecnica, produtividade = self.get_custo()
        return 'Pessoas selecionadas : {}\nCusto monetario da solucao : {}\nCoeficiente tecnico : {}\nProdutividade : {}'.format(self.get_pessoas(), valor, tecnica, produtividade)



def resolver_otimizacao(necessidade_projeto, produtividade_necessaria, pessoas, coeficientes=[40, 60]):
    from ortools.constraint_solver import pywrapcp
    
    search_parameters = pywrapcp.Solver.DefaultSolverParameters()
    search_parameters.trace_search  = True
    search_parameters.trace_propagation  = True
    
    solver = pywrapcp.Solver("Teste", search_parameters)
    
    # declarando as variaveis
    for i in xrange(0, len(pessoas)):
        pessoas[i]['var'] = solver.IntVar(0, 1, 'qt_{}'.format(pessoas[i]['nome']))


    # declarando restricoes
    for hab in necessidade_projeto:
        for chave_hab in necessidade_projeto[hab]:
            valor_minimo = necessidade_projeto[hab][chave_hab]
            for pessoa in pessoas:
                if not pessoa.get(hab, None) is None and not pessoa[hab].get(chave_hab, None) is None:
                    # garante que a habilidade da pessoa atenda ao projeto
                    solver.Add(solver.Max(
                        pessoa['var'] == 0,
                        pessoa['var'] * pessoa[hab][chave_hab] >= valor_minimo
                    ) == True)
                    
                    
                    
    print "Produtividade buscada : {}".format(produtividade_necessaria)
    # garante que a equipe atribuida sera capaz de alcancar a produtividade necessaria
    solver.Add(sum([pessoa['var'] * pessoa['produtividade'] for pessoa in pessoas]) >= produtividade_necessaria)

    #funcao objetivo
    objetivo = solver.Minimize(
        (coeficientes[0] * sum([pessoa['var'] * pessoa['custo_convertido'] for pessoa in pessoas])) +
        (coeficientes[1] * sum([pessoa['var'] * int(pessoa['tot_tecnica']) for pessoa in pessoas])),
    1)




    db = solver.Phase([pessoa['var'] for pessoa in pessoas], solver.CHOOSE_MIN_SIZE_LOWEST_MAX, solver.ASSIGN_CENTER_VALUE)
    solver.TimeLimit(30000)#quero a melhor solucao em 30 segundos de computacao


    solver.NewSearch(db, [objetivo])
    num_solutions = 0

    ultimaSolucao = None

    while solver.NextSolution():
        num_solutions += 1
        
        print [pessoa['var'] for pessoa in pessoas]
        ultimaSolucao = Solucao(num_solutions, pessoas)

    solver.EndSearch()

    if ultimaSolucao!= None:
        print '\n\nMELHOR SOLUCAO ENCONTRADA'
        print ultimaSolucao
    else:
        print 'NENHUMA SOLUCAO ENCONTRADA'
        
    print '\n'

    print num_solutions
    print 'falhas : ', solver.Failures()
    print 'branches : ',solver.Branches()
    print 'tempo em ms : ',solver.WallTime()



resolver_otimizacao(necessidade_projeto, 20, pessoas, coeficientes=[1, 0])

Produtividade buscada : 20
[qt_pessoa_a(1), qt_pessoa_d(0), qt_pessoa_b(0), qt_pessoa_c(1)]


MELHOR SOLUCAO ENCONTRADA
Pessoas selecionadas : ['pessoa_a', 'pessoa_c']
Custo monetario da solucao : 193
Coeficiente tecnico : 134.0
Produtividade : 22


1
falhas :  1
branches :  1
tempo em ms :  2


# Cenário um pouco mais real

### Contexto
Em cada projeto, existe um número de atividades a serem executadas. Cada atividade por sua vez, possui um conjunto de competências necessárias à sua execução, bem como o seu tamanho (mensurado em story points). O somatório do tamanho de cada atividade indica o tamanho do projeto a ser executado.  O projeto conta ainda com um prazo máximo em horas para a sua conclusão.

Quanto aos colaboradores, são aplicáveis as seguintes restrições:
 - Um colaborador deve possuir competência suficiente para atender os requisitos de cada tarefa
 - Um colaborador não pode desempenhar mais de uma atividade por vez
 - Uma atividade comporta apenas um colaborador simultâneamente.
 - Uma vez que uma atividade é atribuída a um colaborador, ela será integralizada por este colaborador.
 - Se um colaborador foi escalado em um projeto, o mesmo deve permanecer no projeto até o seu término, realizando portanto as demais tarefas necessárias?
 
### Objetivo

Determinar quais colaboradores devem atuar em cada atividade no projeto, sem se ocupar com a ordem de execução. A idéia é minimizar o custo do projeto e maximizar a competência da equipe.



#### declarando as variáveis:

In [7]:
projeto = [
    { 
      'nome':'atividade_1',
      'tamanho':13,
      'competencias':{
        'java':2,
        'javascript':4,
        'comprometimento':3
      }
    },
    {
      'nome':'atividade_2',
      'tamanho':8,
      'competencias':{
        'java':1,
        'advpl':4,
      }
    },
    {  
      'nome':'atividade_3',
      'tamanho':21,
      'competencias':{
        'java':4,
        'javascript':2,
      }
    },
    {  
      'nome':'atividade_4',
      'tamanho':3,
      'competencias':{
          'advpl':1,
          'javascript':1,
      }
    },
    {  
      'nome':'atividade_5',
      'tamanho':5,
      'competencias':{
          'java':1,
          'advpl':1,
          'javascript':1,
      }
    },
    
]

pessoas = [
    {'nome':'pessoa_a', 'produtividade':1.2, 'custo_por_hora':113, 'competencias':{'java':5, 'advpl':2, 'javascript':5, 'comprometimento':5}},
    {'nome':'pessoa_b', 'produtividade':1.1, 'custo_por_hora':120, 'competencias':{'java':4, 'advpl':3, 'javascript':4, 'comprometimento':4}},
    {'nome':'pessoa_c', 'produtividade':0.8, 'custo_por_hora':72, 'competencias':{'java':3, 'advpl':3, 'javascript':3, 'comprometimento':3}},
    {'nome':'pessoa_d', 'produtividade':0.9, 'custo_por_hora':70, 'competencias':{'java':2, 'advpl':3, 'javascript':3, 'comprometimento':4}},
    {'nome':'pessoa_e', 'produtividade':0.6, 'custo_por_hora':65, 'competencias':{'java':2, 'advpl':2, 'javascript':2, 'comprometimento':2}},
    {'nome':'pessoa_f', 'produtividade':0.5, 'custo_por_hora':35, 'competencias':{'java':0, 'advpl':1, 'javascript':4, 'comprometimento':1}},
]

In [0]:

def normalizacao()
